# HabitLedger Demo

**AI-powered behavioral money coach** that helps users build lasting financial habits.

📚 **Documentation** (available in the source repository):
- README.md — Overview and setup
- docs/ARCHITECTURE.md — System design and data flow
- docs/OBSERVABILITY.md — Logging and metrics

🔗 **Source Repository**: [github.com/sonalip9/habitledger-agent](https://github.com/sonalip9/habitledger-agent)

---

## What This Demo Shows

1. **Single Interaction** — How the agent analyzes user input and responds
2. **Multiple Scenarios** — Testing different behavioral patterns
3. **Memory Tracking** — How state persists across conversations
4. **Evaluation** — 20 test scenarios with detection metrics

---

## Quick Example

**User says:** "I keep ordering food delivery every evening when stressed."

**HabitLedger responds with:**
- Detected principle: *Habit Loops*
- Explanation of the trigger → routine → reward pattern
- Actionable interventions (meal prep, replace the reward)
- Tracks this pattern for future check-ins

---

## 0. Dependencies & Environment Setup

Install required packages and configure the environment.

**Kaggle Users**: Make sure to add your `GOOGLE_API_KEY` in Kaggle Secrets (Add-ons > Secrets).

In [ ]:
# Install required dependencies (run once per session)
import subprocess
import sys


def install_dependencies():
    """Install required packages for HabitLedger."""
    packages = [
        "google-genai>=0.4.0",
        "python-dotenv",
        "pandas",
    ]
    for package in packages:
        subprocess.check_call(
            [sys.executable, "-m", "pip", "install", "-q", package],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL,
        )
    print("✅ Dependencies installed successfully")


# Uncomment the following line if running on Kaggle or if packages are not installed
# install_dependencies()

## 1. Setup

Load the behavior database and initialize user memory.

In [ ]:
import sys
from pathlib import Path

# First, set up sys.path for imports
notebook_dir = Path.cwd()
if notebook_dir.name == "notebooks":
    project_root = notebook_dir.parent
else:
    project_root = notebook_dir
sys.path.insert(0, str(project_root))

# Now import from config module
from src.config import get_data_path, is_kaggle_environment


# Configure paths
DATA_PATH = get_data_path("behaviour_principles.json")
print(f"📍 Environment: {'Kaggle' if is_kaggle_environment() else 'Local'}")
print(f"📁 Data path: {DATA_PATH}")

In [ ]:
# Import HabitLedger modules
from src.behaviour_engine import analyse_behaviour, load_behaviour_db
from src.coach import generate_session_summary, run_once
from src.config import setup_logging
from src.memory import UserMemory, build_memory_summary
from src.models import Goal

# Configure logging (suppress console output for cleaner notebook)
setup_logging(console=False)

# Load behaviour principles database
behaviour_db = load_behaviour_db(DATA_PATH)
print(f"✅ Loaded {len(behaviour_db.get('principles', []))} behavioural principles")

# Initialize user memory
user_memory = UserMemory(user_id="demo_user")
user_memory.goals = [Goal("Save $500/month"), Goal("Stop impulse buying")]
print(f"✅ Initialized memory for user: {user_memory.user_id}")
print(f"   Goals: {user_memory.goals}")

## 2. Single Interaction

See how the agent processes one user message: analyze behavior, detect principle, suggest interventions.

In [ ]:
# Example user input
user_input = "I keep buying coffee every morning even though I want to save money. It's become such a routine."

print("📥 User Input:")
print(f'"{user_input}"')
print("\n" + "=" * 60)

# Show memory state BEFORE interaction
print("\n📊 Memory State BEFORE:")
print(
    f"   Goals: {len(user_memory.goals)}, Streaks: {len(user_memory.streaks)}, Interventions: {len(user_memory.interventions)}"
)

# Run the agent
response = run_once(
    user_input=user_input, memory=user_memory, behaviour_db=behaviour_db
)

print("\n💬 Agent Response:")
print("-" * 60)
print(response)
print("-" * 60)

# Show memory state AFTER interaction
print("\n📊 Memory State AFTER:")
print(
    f"   Goals: {len(user_memory.goals)}, Streaks: {len(user_memory.streaks)}, Interventions: {len(user_memory.interventions)}"
)

# Show the analysis that happened
print("\n🔍 What the Agent Detected:")
analysis = analyse_behaviour(user_input, user_memory, behaviour_db)
print(f"   Principle: {analysis.get('detected_principle_id', 'None')}")
print(f"   Confidence: {analysis.get('confidence', 0):.0%}")
print(f"   Source: {analysis.get('source', 'unknown').upper()}")

## 3. Multiple Scenarios

Test the agent with different behavioral patterns. Each scenario triggers a different principle from the knowledge base.

📚 The full list of 8 behavioral principles is defined in `data/behaviour_principles.json`.

In [ ]:
# Test scenarios covering different behavioral principles
scenarios = [
    {
        "name": "Loss Aversion",
        "input": "I'm afraid to check my bank account because I might see I've overspent again.",
    },
    {
        "name": "Temptation Bundling",
        "input": "I hate budgeting, it feels like such a chore. I keep putting it off.",
    },
    {
        "name": "Friction Increase",
        "input": "Online shopping is too easy. I just click and buy without thinking.",
    },
    {
        "name": "Micro Habits",
        "input": "Saving $500 a month feels overwhelming. I don't know where to start.",
    },
    {
        "name": "Default Effect",
        "input": "I'm still subscribed to services I never use. I just haven't bothered to cancel them.",
    },
]

# Run through each scenario
print("=" * 60)
for i, scenario in enumerate(scenarios, 1):
    print(f"\n📌 SCENARIO {i}: {scenario['name']}")
    print(f"User: {scenario['input']}")
    print("-" * 60)

    response = run_once(
        user_input=scenario["input"], memory=user_memory, behaviour_db=behaviour_db
    )

    print(response)
    print("=" * 60)

# Show memory summary after all scenarios
print("\n📊 MEMORY SUMMARY")
print(build_memory_summary(user_memory))

## 4. Memory Visualization

See how the agent tracks state over time: goals, streaks, struggles, and conversation history.

📚 The full data model is documented in `docs/ARCHITECTURE.md`.

In [ ]:
import json
import pandas as pd

print("📊 MEMORY STATE")
print("=" * 60)

# Summary
print(build_memory_summary(user_memory))

# Conversation history table
if user_memory.conversation_history:
    print("\n💬 Conversation History:")
    conv_data = []
    for turn in user_memory.conversation_history[-6:]:  # Last 6 turns
        conv_data.append(
            {
                "Role": turn.role.value.upper(),
                "Content": (
                    turn.content[:50] + "..."
                    if len(turn.content) > 50
                    else turn.content
                ),
            }
        )
    print(pd.DataFrame(conv_data).to_string(index=False))

# Full JSON state (collapsed for inspection)
print("\n📄 Full Memory State (JSON):")
print(json.dumps(user_memory.to_dict(), indent=2, default=str)[:1000] + "...")

## 5. Comprehensive Evaluation

Run 20 test scenarios to measure detection accuracy and response quality.

📚 Detailed evaluation methodology is documented in `docs/EVALUATION_RESULTS.md`.

**Note**: This evaluation makes multiple LLM calls. Adjust `MAX_SCENARIOS` below to limit API usage.

In [ ]:
import time

# Configuration: Set to a lower number to reduce API calls
MAX_SCENARIOS = 20  # Set to 5 or 10 for faster runs with fewer API calls

# Fresh memory for evaluation
eval_memory = UserMemory(user_id="eval_user")

# Test scenarios covering all 8 principles
test_scenarios = [
    # Loss Aversion
    "I'm afraid to check my savings account because I might see I've spent too much",
    "I broke my 30-day no-spending streak yesterday and feel terrible about it",
    "I regret buying that expensive gadget last week",
    # Habit Loops
    "Every evening after work I automatically order food delivery when I'm stressed",
    "Whenever I feel bored, I start browsing shopping apps",
    "I always grab coffee at the same cafe during my lunch break",
    # Commitment Devices
    "I need help sticking to my budget, my willpower isn't enough",
    "I keep forgetting to transfer money to my savings account",
    "It's hard to resist temptation when sales pop up",
    # Temptation Bundling
    "Reviewing my expenses is so boring and tedious",
    "I dread looking at my budget spreadsheet",
    # Friction Reduction
    "Tracking my expenses takes too many steps and is confusing",
    "Setting up automatic savings is too complicated",
    # Friction Increase
    "One-click ordering makes it too easy to impulse buy",
    "I keep ordering food delivery because the app is right on my phone",
    "Online shopping is instant, I don't even think about it",
    # Default Effect
    "I forget to save money each month, I need to automate it",
    "I'm still paying for subscriptions I never use",
    # Micro Habits
    "Saving $1000 a month feels overwhelming and impossible",
    "My financial goals are too big, I don't know where to start",
]

# Limit scenarios based on configuration
scenarios_to_run = test_scenarios[:MAX_SCENARIOS]

print(f"🧪 Running {len(scenarios_to_run)} evaluation scenarios...\n")

# Run evaluation
results = []
start_time = time.time()

for idx, prompt in enumerate(scenarios_to_run, 1):
    analysis = analyse_behaviour(prompt, eval_memory, behaviour_db)
    response = run_once(prompt, eval_memory, behaviour_db)

    results.append(
        {
            "ID": idx,
            "Prompt": prompt[:50] + "..." if len(prompt) > 50 else prompt,
            "Principle": analysis.get("detected_principle_id", "None"),
            "Source": analysis.get("source", "unknown"),
            "Response Len": len(response),
        }
    )

eval_duration = time.time() - start_time

# Display results
df_results = pd.DataFrame(results)
print(df_results.to_string(index=False))

# Summary metrics
print(f"\n📈 Evaluation Metrics:")
print(f"  • Total scenarios: {len(scenarios_to_run)}")
print(f"  • Avg response length: {df_results['Response Len'].mean():.0f} chars")
print(f"  • Total time: {eval_duration:.2f}s")
print(f"  • Avg time per scenario: {eval_duration/len(scenarios_to_run):.3f}s")

### Evaluation Analysis

In [ ]:
# Principle distribution
print("📊 PRINCIPLE DISTRIBUTION")
print("-" * 40)
principle_counts = df_results["Principle"].value_counts()
for principle, count in principle_counts.items():
    pct = (count / len(df_results)) * 100
    print(f"  {principle:20s} | {count:2d} ({pct:4.1f}%)")

# Source distribution (LLM vs Keyword)
print("\n📊 DETECTION SOURCE")
print("-" * 40)
source_counts = df_results["Source"].value_counts()
for source, count in source_counts.items():
    pct = (count / len(df_results)) * 100
    print(f"  {source.upper():10s}: {count:2d} ({pct:.1f}%)")  # type: ignore

# Response quality
print("\n📊 RESPONSE QUALITY")
print("-" * 40)
print(f"  Mean length:   {df_results['Response Len'].mean():.0f} chars")
print(f"  Min length:    {df_results['Response Len'].min():.0f} chars")
print(f"  Max length:    {df_results['Response Len'].max():.0f} chars")

# Coverage check
detected_principles = set(df_results["Principle"].unique()) - {"None", None}
all_principles = {
    "loss_aversion",
    "habit_loops",
    "commitment_devices",
    "temptation_bundling",
    "friction_reduction",
    "friction_increase",
    "default_effect",
    "micro_habits",
}
coverage = len(detected_principles) / len(all_principles) * 100
print(
    f"\n📊 COVERAGE: {len(detected_principles)}/{len(all_principles)} principles ({coverage:.0f}%)"
)

## 6. Session Summary

Generate a final summary of the evaluation session.

In [ ]:
print("📊 SESSION SUMMARY")
print("=" * 60)

# Memory summary
print(build_memory_summary(eval_memory))

# Session summary with encouragement
print(generate_session_summary(eval_memory))

---

## Key Takeaways

This demo showcased HabitLedger's core capabilities:

| Feature | Description |
|---------|-------------|
| **LLM-Powered Analysis** | Gemini models via Google ADK detect behavioral patterns |
| **Keyword Fallback** | Reliable detection when LLM is unavailable |
| **Persistent Memory** | Goals, streaks, and struggles tracked over time |
| **Knowledge Base** | 8 behavioral principles with interventions |
| **Observability** | Structured logging for decision transparency |

---

### Running Locally

```bash
# Clone the repository
git clone https://github.com/sonalip9/habitledger-agent.git
cd habitledger-agent

# Install dependencies
pip install -r requirements.txt

# Set up your API key
cp .env.example .env
# Edit .env and add your GOOGLE_API_KEY

# Run CLI mode
python -m src.coach

# Run with ADK
python -m src.habitledger_adk.runner
```

📚 **More documentation in the source repository:**
- README.md — Setup and overview
- docs/ARCHITECTURE.md — System design
- docs/DEVELOPMENT.md — Development guide
- docs/OBSERVABILITY.md — Logging and metrics

---

*HabitLedger — Building better financial habits, one conversation at a time.* 💰